Using any OpenAI model or Groq (llama 70b) model, solve the aisle mapping problem.  OpenAI code is provided here. You will do the usual 3 steps in terms of mounting the Google drive, your API key and install one of the LLM models, and import pandas.  Your goal is to do a model comparison and validation.
**The code below works for OpenAI gpt-4o model.  Have not tested it on Llama model.  Also, this code with the dataset  may blow your budget if you are not careful with the size of test dataset, so exercise caution.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install -r "/content/drive/MyDrive/LLM/requirements.txt"

In [ ]:
import os  ## step-3
# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLM/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set
!echo $OPENAI_API_KEY

In [ ]:
import openai
client = openai.OpenAI() #step-4

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
import pandas as pd
df = pd.read_excel('Aisle-Mapping.xlsx')

In [ ]:
groceries = df['Grocery ITEM'].dropna().tolist()
aisles = df['Aisle Category'].dropna().tolist()

In [ ]:
import json


# Function to get the best matches for a batch of keywords using chat completion
def get_best_matches_batch(grocery_batch, aisle_list):
    prompt = "Match each grocery item in the  grocery list with the most appropriate aisle category from the provided list of aisle categories.\n\n"
    prompt += "The grocery list items are separated by commas. The list of aisles are also separated by commas. \n\n"
    prompt += "List of grocery items to match:\n" + "\n"+ "\n"+separator.join(grocery_batch)+  "\n\n"
    prompt += "List of provided aisle categories:\n" + "\n"+ "\n"+separator.join(aisle_list)+ "\n\n"
    prompt += "If an appropriate aisle category is not to be found in the list, use Other \n\n"
    prompt += "Return the matches in the format {'grocery_item': 'aisle_category'} \n\n"
    prompt+="You must absolutely make sure that each grocery item is mapped to an aisle category"
    #print(prompt)
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that matches grocery items to aisle categories based on a typical grocery store or a supermarket in the USA."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=4096,  # Increase the max_tokens to generate more matches
    temperature=0.0
    )
    matches = response.choices[0].message.content
    # Convert the matches to a JSON-formatted string
    print(matches)
    #json_matches = json.dumps({item.split(' -> ')[0]: item.split(' -> ')[1] for item in matches.split('\n') if item and len(item.split(' -> ')) > 1})
    json_matches = json.dumps({item.split(': ')[0].strip(): item.split(': ')[1].strip() for item in matches.split('\n') if item and len(item.split(': ')) > 1})
      #print("JSON Matches:", json_matches)
    return json_matches

In [ ]:
grocery_batch = groceries[0: batch_size]
batch_matches = get_best_matches_batch(grocery_batch, aisles)
#print(batch_matches)




{
'tropical fruit': 'Produce',
'whole milk': 'Dairy',
'pip fruit': 'Produce',
'other vegetables': 'Produce',
'rolls/buns': 'Bakery',
'pot plants': 'Garden',
'citrus fruit': 'Produce',
'beef': 'Meat',
'frankfurter': 'Meat',
'chicken': 'Meat',
'butter': 'Dairy',
'fruit/vegetable juice': 'Beverages',
'packaged fruit/vegetables': 'Produce',
'chocolate': 'Candy',
'specialty bar': 'Candy',
'butter milk': 'Dairy',
'bottled water': 'Beverages',
'yogurt': 'Dairy',
'sausage': 'Meat',
'brown bread': 'Bakery',
'hamburger meat': 'Meat',
'root vegetables': 'Produce',
'pork': 'Meat',
'pastry': 'Bakery',
'canned beer': 'Alcohol',
'berries': 'Produce',
'coffee': 'Beverages',
'misc. beverages': 'Beverages',
}
